# Install

In [ ]:
!git clone https://github.com/PixArt-alpha/PixArt-alpha.git

In [ ]:
%cd PixArt-alpha

In [ ]:
!pip install torch==2.0.0+cu117 torchvision==0.15.1+cu117 torchaudio==2.0.1 --index-url https://download.pytorch.org/whl/cu117
!pip install -r requirements.txt

## Download model

In [ ]:
!python tools/download.py --model_names "PixArt-XL-2-512x512.pth"

## Extract features

In [ ]:
!python /workspace/PixArt-alpha/tools/extract_features.py \
    --json_path "/workspace/pixart-pokemon/data_info.json" \
    --t5_save_root "/workspace/pixart-pokemon/caption_feature_wmask" \
    --vae_save_root "/workspace/pixart-pokemon/img_vae_features" \
    --pretrained_models_dir "/workspace/PixArt-alpha/output/pretrained_models" \
    --dataset_root "/workspace/pixart-pokemon"

## Train model

In [ ]:
!python \
    scripts/train.py \
    /workspace/pixart-pokemon/PixArt_xl2_img512_internal.py \
    --work-dir output/trained_model

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
2023-11-30 12:27:22,610 - PixArt - INFO - Config: 
data_root = '/workspace'
data = dict(
    type='InternalData',
    root='/workspace/pixart-pokemon',
    image_list_json=['data_info.json'],
    transform='default_train',
    load_vae_feat=True)
image_size = 512
train_batch_size = 38
eval_batch_size = 16
use_fsdp = False
valid_num = 2000
model = 'PixArt_XL_2'
aspect_ratio_type = None
multi_scale = False
lewei_scale = 1.0
num_workers = 10
train_sampling_steps = 1000
eval_sampling_steps = 200
num_epochs = 200
gradient_accumulation_steps = 1
grad_checkpointing = True
gradient_clip = 0.01
gc_step = 1
auto_lr = dict(rule='sqrt')
optimizer = dict(type='AdamW', lr=2e-05, weight_decay=0.03, eps=1e-10)
lr_schedule = 'constant'
lr_schedule_args = dict(num_warmup_steps=1000)
save_image_epochs = 1
save_model_e